## Challenge 0: Install dependencies

In [ ]:
! conda install 'ase==3.24.0' --yes
! pip install 'matgl==1.2.1' mace_torch

## Challenge 1: Build a forcefield based equation of state maker

Using `atomate2.common.flows.eos.CommonEosMaker` as a guide, or starting from scratch, write an equation of state workflow for any of the MLIPs available in `atomate2`, or any ASE classical forcefield.

Key questions:
- How do we ensure the EOS is run at fixed volume?
- What should the inputs to the maker look like?


<details>
<summary><b>Hint</b></summary>
<br>
There are MLIP EOS makers in `atomate2` - you can refer to these for guidance.
</details>

### When you are confident in your workflow:
Run it localy (`run_locally`) for the Materials Project structure below ([mp-22526](https://next-gen.materialsproject.org/materials/mp-22526)):


<details>
<summary><b>Reminder</b></summary>
<br>
When you define "complex" (usually mutable) objects as defaults in a python `dataclass`, you need to use the `field` function to define them.
For example, you might see a dataclass like this:

```python
from dataclasses import dataclass, field
@dataclass
class purplePeopleEater:

    eyes : int = 1
    horns : str = "single"
    abilities : dict[str,str] = field(default_factory = dict)
```
which means that a new "instance" of `purplePeopleEater()` will have default attributes of `eyes = 1`, `horns = "single"` and an empty dictionary of abilities.
The line:

```python
abilities : dict[str,str] = field(default_factory = dict)
```
indicates that when the class is called as `purplePeopleEater()`, the field will be given an empty dictionary as default.
</details>

In [33]:
from pymatgen.core import Structure

test_structure = Structure.from_str(
"""mp-22526
1.0
   2.6937121874714256    0.0000000000000000    4.1079559895387510
   1.2267470851777738    2.3981611576247186    4.1079559895387510
   0.0000000000000000    0.0000000000000000    4.9123704799999999
Li Co O
1 1 2
direct
   0.0000000000000000    0.0000000000000000    0.0000000000000000 Li
   0.5000000000000000    0.5000000000000000    0.5000000000000000 Co
   0.7599932000000000    0.7599932000000000    0.7599932000000000 O
   0.2400068000000000    0.2400068000000000    0.2400068000000000 O
""",
fmt = "poscar"
)

In [3]:
test_structure.scale_lattice(1.1*test_structure.volume)

Structure Summary
Lattice
    abc : 5.070942789493697 5.070942789493698 5.0709423662588495
 angles : 33.25407938000001 33.25407937999999 33.254068270000005
 volume : 34.907066536121576
      A : 2.7806655278892265 0.0 4.240561283170737
      B : 1.266346622723074 2.4755740766758194 4.240561283170737
      C : 0.0 0.0 5.0709423662588495
    pbc : True True True
PeriodicSite: Li (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: Co (2.024, 1.238, 6.776) [0.5, 0.5, 0.5]
PeriodicSite: O (3.076, 1.881, 10.3) [0.76, 0.76, 0.76]
PeriodicSite: O (0.9713, 0.5942, 3.253) [0.24, 0.24, 0.24]

In [79]:
from __future__ import annotations
from dataclasses import dataclass, field
import numpy as np

from jobflow import Maker, Flow, run_locally, job
from atomate2.forcefields.jobs import ForceFieldRelaxMaker
from pymatgen.analysis.eos import EOS

@job
def eos_fit(volumes : list[float], energies : list[float],eos_model : str = "vinet"):
    fitted_eos = EOS(eos_model).fit(volumes,energies)
    return {
        "eos_params": {k : getattr(fitted_eos,k,None) for k in ("e0","v0","b0","b1")},
        "eos_model": eos_model,
        "energies": energies,
        "volumes": volumes,
    }

@dataclass
class ForceFieldEos(Maker):
    name : str = "MACE EOS"
    eos_relaxer : Maker = field(
        default_factory = lambda : ForceFieldRelaxMaker(
            force_field_name = "MACE",
            relax_cell = False,
        )
    )
    linear_strain : tuple[float,float] = (-0.05, 0.05)
    num_frames : int = 6
    eos_model : str = "vinet"
    
    """
    Fill out any other fields you might need here.
    """
    def make(
        self,
        structure : Structure,
        prev_dir : str | None = None # this means that prev_dir can be a string, or None, but defaults to None
    ) -> Flow: # this means that `make` returns a `jobflow` `Flow`
        
        jobs = []
        structure_dict = structure.as_dict()

        energies = []
        volumes = []

        strains = np.linspace(self.linear_strain[0],self.linear_strain[1],self.num_frames)
        for strain in strains:
            strained_structure = Structure.from_dict(structure_dict)
            strained_structure = strained_structure.scale_lattice((1. + strain)*structure.volume)
            job = self.eos_relaxer.make(strained_structure)
            job.append_name(f" {strain:.4f}")
            jobs.append(job)

            energies.append(job.output.output.energy)
            volumes.append(job.output.structure.volume)

        fit_job = eos_fit(volumes, energies, eos_model = self.eos_model)
        
        return Flow(jobs + [fit_job])
    
flow = ForceFieldEos().make(test_structure)
response = run_locally(flow)

2025-03-19 16:29:43,658 INFO Started executing jobs locally


INFO:jobflow.managers.local:Started executing jobs locally


2025-03-19 16:29:43,664 INFO Starting job - Force field relax -0.0500 (17781868-f16a-4840-aab3-bac557a89025)


INFO:jobflow.core.job:Starting job - Force field relax -0.0500 (17781868-f16a-4840-aab3-bac557a89025)


Using medium MPA-0 model as default MACE-MP model, to use previous (before 3.10) default model please specify 'medium' as model argument
Using Materials Project MACE for MACECalculator with /home/jovyan/.cache/mace/macempa0mediummodel
Using float32 for MACECalculator, which is faster but less accurate. Recommended for MD. Use float64 for geometry optimization.
Default dtype float32 does not match model dtype float64, converting models to float32.


...

INFO:jobflow.core.job:Starting job - eos_fit (3e263e2f-1db3-4e89-9ed4-98de5ca8522c)


2025-03-19 16:29:49,796 INFO Finished job - eos_fit (3e263e2f-1db3-4e89-9ed4-98de5ca8522c)


INFO:jobflow.core.job:Finished job - eos_fit (3e263e2f-1db3-4e89-9ed4-98de5ca8522c)


2025-03-19 16:29:49,797 INFO Finished executing jobs locally


INFO:jobflow.managers.local:Finished executing jobs locally


In [76]:
uuid_to_job_name = {
    job.uuid : job.name
    for job in flow.jobs
}
flow_output = {
    name : response[uuid][1].output
    for uuid, name in uuid_to_job_name.items()
}

In [78]:
flow_output['eos_fit']

{'eos_params': {'e0': -22.50060632220471,
  'v0': 33.83600483683379,
  'b0': 1.0605797367184642,
  'b1': 6.253488726999332},
 'eos_model': 'vinet',
 'energies': [-22.219335556030273,
  -22.317123413085938,
  -22.390911102294922,
  -22.443735122680664,
  -22.47804832458496,
  -22.496286392211914],
 'volumes': [30.147012008468636,
  30.781685945489027,
  31.416359882509422,
  32.0510338195298,
  32.68570775655021,
  33.320381693570596]}

### Challenge 1a: Build a postprocessing job for the EOS flow

Using `pymatgen.analysis.eos.EOS` and `matplotlib`, write a function that can be used to fit and plot the output of the EOS flow. How could you incorporate this into the flow as a job?

## Challenge 2: Build a custom relax maker using ASE

Just like `pymatgen`, ASE also has an interface to VASP, among other electronic structure codes.
You can get a sense for how to implement a VASP maker using ASE by using the much simpler EMT calculator from ASE.
This is basically a low-accuracy interatomic potential for alloys/intermetallics.
Build a simple EMT relax maker using `atomate2.ase.jobs.AseMaker`.

The structure of `AseMaker` requires a `run_ase` function and a `calculator` attribute to be defined.
We've defined the `run_ase` function for you, but you still need to call it from `make`.

When you're ready to run it, `run_locally` on the intermetallic structure below ([mp-1228912](https://next-gen.materialsproject.org/materials/mp-1228912)):

In [85]:
intermetallic = Structure.from_str("""mp-1228912
1.0
   3.1077110000000001    0.0000000000000000    0.0000000000000000
   0.0000000000000000    3.1077110000000001    0.0000000000000000
   0.0000000000000000    0.0000000000000000    5.7859559999999997
Al Cu Pd
1 1 2
direct
   0.0000000000000000    0.0000000000000000    0.0000000000000000 Al
   0.0000000000000000    0.0000000000000000    0.5000000000000000 Cu
   0.5000000000000000    0.5000000000000000    0.2450020000000000 Pd
   0.5000000000000000    0.5000000000000000    0.7549979999999999 Pd
""",
fmt="poscar"
)

In [100]:
from __future__ import annotations
from dataclasses import dataclass, field
from jobflow import job, Job

from ase.calculators.calculator import Calculator
from ase.calculators.emt import EMT

from atomate2.ase.schemas import AseResult
from atomate2.ase.jobs import AseMaker, AseRelaxMaker

from pymatgen.io.ase import AseAtomsAdaptor

@dataclass
class AseEMTStatic(AseMaker):
    name : str = "ASE EMT Static" # this is required!
    """
    Fill out any other fields you might need here.
    """

    def calculator(self) -> Calculator:
        """ASE EMT calculator."""
        return EMT()
    
    @job
    def make(
        self,
        structure : Structure,
        prev_dir : str | None = None,
    ) -> Job:
        
        return self.run_ase(structure)
 
    def run_ase(
        self,
        structure: Structure,
        prev_dir: str | None = None,
    ) -> AseResult:
        
        adaptor = AseAtomsAdaptor()
        atoms = adaptor.get_atoms(structure)
        atoms.calc = self.calculator()
        toten = atoms.get_potential_energy()

        final_structure = adaptor.get_structure(atoms)

        return {
            "final_structure": final_structure,
            "final_total_energy": toten,
        }

@dataclass
class AseEMTRelax(AseRelaxMaker):
    name : str = "ASE EMT relaxer"

    @property
    def calculator(self):
        return EMT()

@dataclass
class AseEMTEos(ForceFieldEos):

    name : str = "ASE EMT EOS"
    eos_relaxer : Maker = field(
        default_factory = lambda : AseEMTRelax(
            relax_cell = False,
        )
    )

In [95]:
intermet_relax_job = AseEMTRelax().make(intermetallic)
intermet_resp = run_locally(intermet_relax_job)

2025-03-19 17:24:13,163 INFO Started executing jobs locally


INFO:jobflow.managers.local:Started executing jobs locally


2025-03-19 17:24:13,168 INFO Starting job - ASE EMT relaxer (45524209-628b-4b48-8f35-252903c06fb8)


INFO:jobflow.core.job:Starting job - ASE EMT relaxer (45524209-628b-4b48-8f35-252903c06fb8)
/opt/conda/lib/python3.12/site-packages/atomate2/ase/utils.py:368: FutureWarning: Please use atoms.calc = calc
  atoms.set_calculator(self.calculator)


2025-03-19 17:24:13,636 INFO Finished job - ASE EMT relaxer (45524209-628b-4b48-8f35-252903c06fb8)


INFO:jobflow.core.job:Finished job - ASE EMT relaxer (45524209-628b-4b48-8f35-252903c06fb8)


2025-03-19 17:24:13,638 INFO Finished executing jobs locally


INFO:jobflow.managers.local:Finished executing jobs locally


### Challenge 2a: Build an EMT EOS maker

Adapt your EOS flow to use the EMT relax maker you just developed.

In [102]:
eos_job = AseEMTEos().make(intermetallic)
eos_emt_resp = run_locally(eos_job)

emt_uuid_to_job_name = {
    job.uuid : job.name
    for job in eos_job.jobs
}
emt_eos_output = {
    name : eos_emt_resp[uuid][1].output
    for uuid, name in emt_uuid_to_job_name.items()
}
emt_eos_output['eos_fit']

2025-03-19 17:30:56,253 INFO Started executing jobs locally


INFO:jobflow.managers.local:Started executing jobs locally


2025-03-19 17:30:56,260 INFO Starting job - ASE EMT relaxer -0.0500 (b44b0ea2-840f-4445-abf2-ea6b927e8855)


INFO:jobflow.core.job:Starting job - ASE EMT relaxer -0.0500 (b44b0ea2-840f-4445-abf2-ea6b927e8855)


2025-03-19 17:30:56,299 INFO Finished job - ASE EMT relaxer -0.0500 (b44b0ea2-840f-4445-abf2-ea6b927e8855)


...

2025-03-19 17:30:56,712 INFO Finished job - eos_fit (ad0896aa-341e-4504-a319-3cd697a2e823)


INFO:jobflow.core.job:Finished job - eos_fit (ad0896aa-341e-4504-a319-3cd697a2e823)


2025-03-19 17:30:56,713 INFO Finished executing jobs locally


INFO:jobflow.managers.local:Finished executing jobs locally


{'eos_params': {'e0': 0.17872386719312902,
  'v0': 58.821931806959974,
  'b0': 0.6999071343385862,
  'b1': 3.331984592394601},
 'eos_model': 'vinet',
 'energies': [0.4053215329904445,
  0.3213887652213243,
  0.25828357383787814,
  0.21464135590433653,
  0.18852339176691313,
  0.17886140600517475],
 'volumes': [53.0859974652209,
  54.20359741185712,
  55.32119735849335,
  56.4387973051296,
  57.556397251765816,
  58.67399719840205]}

### Challenge 2b: Modifying the output of the EMT job

What other useful information could be included in either the output of `run_ase` or `make`?
Modify the output of your EMT maker to include the forces or other useful information.